In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
#from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from data_loader import DataLoader
import numpy as np
import os

Using TensorFlow backend.


In [2]:
class Pix2Pix():
    def __init__(self):
        # Input shape
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'place365'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])

        self.combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = BatchNormalization(momentum=0.8)(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model([img_A, img_B], validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            
            
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                #imgs_B = np.expand_dims(imgs_B, axis=0)
                fake_A = self.generator.predict(imgs_B)

                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.25 * np.add(d_loss_real, d_loss_fake)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])
                for i in range(1):
                    g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])
                    
                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        elapsed_time))

                # If at save interval => save generated image samples
                #if batch_i % sample_interval == 0:
                    #self.sample_images(epoch, batch_i)

            self.save(epoch)
            print("weights saved...")
    def sample_images(self, epoch, batch_i):
        os.makedirs('images/UNET-%s-%d' % (epoch,batch_i), exist_ok=True)
        imgs_A, imgs_B = self.data_loader.load_data(batch_size=40, is_testing=True)
        fake_A = gan.generator.predict(imgs_B)
        fake_A = 0.5 * fake_A + 0.5
        f = 0
        for img in fake_A:
            imsave('images/UNET-%s-%d/turing%d.png' % (epoch,batch_i, f))
            f = f+1
            
    def sample_imagesOLD(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()
        self.save("epoch-"+str(epoch))
        
    def save(self,ep):
        self.combined.save_weights("./saved_model/modelW_epoch_%d.h5" % (ep))
        self.generator.save_weights("./saved_model/genW_epoch_%d.h5" % (ep))
        self.discriminator.save_weights("./saved_model/disW_epoch_%d.h5" % (ep))
    def load(self,ep):
        self.combined.load_weights("./saved_model/modelW_epoch_%d.h5" % (ep))
        self.generator.load_weights("./saved_model/genW_epoch_%d.h5" % (ep))
        self.discriminator.load_weights("./saved_model/disW_epoch_%d.h5" % (ep))

In [3]:
gan = Pix2Pix()
#gan.load(12)

In [4]:
gan.train(epochs=6, batch_size=30, sample_interval=500)

C:\Users\miche\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/6] [Batch 0/1166] [D loss: 5.536636, acc:   7%] [G loss: 52.355873] time: 0:00:10.877022
[Epoch 0/6] [Batch 1/1166] [D loss: 8.450064, acc:   4%] [G loss: 41.635887] time: 0:00:13.413240
[Epoch 0/6] [Batch 2/1166] [D loss: 3.651955, acc:   6%] [G loss: 35.905830] time: 0:00:15.867647
[Epoch 0/6] [Batch 3/1166] [D loss: 1.768165, acc:  13%] [G loss: 25.582241] time: 0:00:18.310147
[Epoch 0/6] [Batch 4/1166] [D loss: 0.727253, acc:  17%] [G loss: 22.398815] time: 0:00:20.762559
[Epoch 0/6] [Batch 5/1166] [D loss: 0.681952, acc:  16%] [G loss: 21.468267] time: 0:00:23.201039
[Epoch 0/6] [Batch 6/1166] [D loss: 0.492077, acc:  19%] [G loss: 17.896746] time: 0:00:25.645534
[Epoch 0/6] [Batch 7/1166] [D loss: 0.413111, acc:  21%] [G loss: 17.840527] time: 0:00:28.110910
[Epoch 0/6] [Batch 8/1166] [D loss: 0.368097, acc:  21%] [G loss: 18.859703] time: 0:00:30.560361
[Epoch 0/6] [Batch 9/1166] [D loss: 0.346501, acc:  22%] [G loss: 17.724945] time: 0:00:33.017791
[Epoch 0/6] [Batch 1

[Epoch 0/6] [Batch 83/1166] [D loss: 0.189627, acc:  24%] [G loss: 12.034080] time: 0:03:46.279081
[Epoch 0/6] [Batch 84/1166] [D loss: 0.185819, acc:  25%] [G loss: 13.095265] time: 0:03:48.908020
[Epoch 0/6] [Batch 85/1166] [D loss: 0.184006, acc:  23%] [G loss: 11.473321] time: 0:03:51.532004
[Epoch 0/6] [Batch 86/1166] [D loss: 0.164052, acc:  25%] [G loss: 12.675610] time: 0:03:54.139033
[Epoch 0/6] [Batch 87/1166] [D loss: 0.142822, acc:  24%] [G loss: 12.146696] time: 0:03:56.750052
[Epoch 0/6] [Batch 88/1166] [D loss: 0.158829, acc:  25%] [G loss: 11.322322] time: 0:03:59.358078
[Epoch 0/6] [Batch 89/1166] [D loss: 0.182505, acc:  25%] [G loss: 13.005655] time: 0:04:01.967103
[Epoch 0/6] [Batch 90/1166] [D loss: 0.202849, acc:  25%] [G loss: 12.394142] time: 0:04:04.577124
[Epoch 0/6] [Batch 91/1166] [D loss: 0.233730, acc:  26%] [G loss: 12.004974] time: 0:04:07.188142
[Epoch 0/6] [Batch 92/1166] [D loss: 0.210206, acc:  26%] [G loss: 11.814261] time: 0:04:09.797166
[Epoch 0/6

[Epoch 0/6] [Batch 166/1166] [D loss: 0.147668, acc:  24%] [G loss: 9.568731] time: 0:07:22.960687
[Epoch 0/6] [Batch 167/1166] [D loss: 0.157220, acc:  24%] [G loss: 10.636823] time: 0:07:25.568745
[Epoch 0/6] [Batch 168/1166] [D loss: 0.149075, acc:  26%] [G loss: 11.125816] time: 0:07:28.185716
[Epoch 0/6] [Batch 169/1166] [D loss: 0.143862, acc:  25%] [G loss: 11.436572] time: 0:07:30.793743
[Epoch 0/6] [Batch 170/1166] [D loss: 0.148574, acc:  24%] [G loss: 10.866836] time: 0:07:33.406756
[Epoch 0/6] [Batch 171/1166] [D loss: 0.162360, acc:  24%] [G loss: 10.409470] time: 0:07:36.015781
[Epoch 0/6] [Batch 172/1166] [D loss: 0.173115, acc:  24%] [G loss: 9.929174] time: 0:07:38.620815
[Epoch 0/6] [Batch 173/1166] [D loss: 0.174024, acc:  23%] [G loss: 11.087582] time: 0:07:41.226847
[Epoch 0/6] [Batch 174/1166] [D loss: 0.151853, acc:  26%] [G loss: 11.171860] time: 0:07:43.834905
[Epoch 0/6] [Batch 175/1166] [D loss: 0.136870, acc:  24%] [G loss: 10.290235] time: 0:07:46.442900
[E

[Epoch 0/6] [Batch 248/1166] [D loss: 0.128023, acc:  26%] [G loss: 11.350274] time: 0:10:57.082169
[Epoch 0/6] [Batch 249/1166] [D loss: 0.131149, acc:  27%] [G loss: 11.569396] time: 0:10:59.690196
[Epoch 0/6] [Batch 250/1166] [D loss: 0.140482, acc:  25%] [G loss: 8.658991] time: 0:11:02.294233
[Epoch 0/6] [Batch 251/1166] [D loss: 0.140167, acc:  24%] [G loss: 11.630568] time: 0:11:04.895310
[Epoch 0/6] [Batch 252/1166] [D loss: 0.135398, acc:  26%] [G loss: 12.683186] time: 0:11:07.501310
[Epoch 0/6] [Batch 253/1166] [D loss: 0.142902, acc:  25%] [G loss: 11.434979] time: 0:11:10.102388
[Epoch 0/6] [Batch 254/1166] [D loss: 0.141974, acc:  27%] [G loss: 13.468713] time: 0:11:12.697417
[Epoch 0/6] [Batch 255/1166] [D loss: 0.133871, acc:  25%] [G loss: 11.039437] time: 0:11:15.297465
[Epoch 0/6] [Batch 256/1166] [D loss: 0.125960, acc:  26%] [G loss: 12.802035] time: 0:11:17.904495
[Epoch 0/6] [Batch 257/1166] [D loss: 0.132644, acc:  26%] [G loss: 11.594126] time: 0:11:20.504542
[

...

[Epoch 5/6] [Batch 792/1166] [D loss: 0.038226, acc:  48%] [G loss: 10.679143] time: 4:46:42.888882
[Epoch 5/6] [Batch 793/1166] [D loss: 0.103066, acc:  27%] [G loss: 8.993085] time: 4:46:45.479922
[Epoch 5/6] [Batch 794/1166] [D loss: 0.032585, acc:  48%] [G loss: 10.839577] time: 4:46:48.084988
[Epoch 5/6] [Batch 795/1166] [D loss: 0.035627, acc:  47%] [G loss: 9.882393] time: 4:46:50.677026
[Epoch 5/6] [Batch 796/1166] [D loss: 0.039199, acc:  45%] [G loss: 9.783142] time: 4:46:53.271090
[Epoch 5/6] [Batch 797/1166] [D loss: 0.187413, acc:  21%] [G loss: 10.697152] time: 4:46:55.867148
[Epoch 5/6] [Batch 798/1166] [D loss: 0.011099, acc:  49%] [G loss: 10.782581] time: 4:46:58.456256
[Epoch 5/6] [Batch 799/1166] [D loss: 0.020233, acc:  49%] [G loss: 11.729836] time: 4:47:01.055276
[Epoch 5/6] [Batch 800/1166] [D loss: 0.007486, acc:  49%] [G loss: 12.510273] time: 4:47:03.646348
[Epoch 5/6] [Batch 801/1166] [D loss: 0.022287, acc:  49%] [G loss: 10.196583] time: 4:47:06.244401
[Ep

[Epoch 5/6] [Batch 875/1166] [D loss: 0.022301, acc:  48%] [G loss: 12.156031] time: 4:50:18.418567
[Epoch 5/6] [Batch 876/1166] [D loss: 0.066338, acc:  38%] [G loss: 9.357448] time: 4:50:21.019612
[Epoch 5/6] [Batch 877/1166] [D loss: 0.159615, acc:  22%] [G loss: 9.935677] time: 4:50:23.612711
[Epoch 5/6] [Batch 878/1166] [D loss: 0.017534, acc:  48%] [G loss: 10.605831] time: 4:50:26.202756
[Epoch 5/6] [Batch 879/1166] [D loss: 0.252801, acc:   7%] [G loss: 9.010248] time: 4:50:28.797815
[Epoch 5/6] [Batch 880/1166] [D loss: 0.059518, acc:  45%] [G loss: 10.892118] time: 4:50:31.392876
[Epoch 5/6] [Batch 881/1166] [D loss: 0.091042, acc:  38%] [G loss: 8.776191] time: 4:50:33.981954
[Epoch 5/6] [Batch 882/1166] [D loss: 0.183652, acc:  11%] [G loss: 9.354677] time: 4:50:36.577015
[Epoch 5/6] [Batch 883/1166] [D loss: 0.017531, acc:  49%] [G loss: 9.671259] time: 4:50:39.167090
[Epoch 5/6] [Batch 884/1166] [D loss: 0.032859, acc:  48%] [G loss: 9.485326] time: 4:50:41.761154
[Epoch 

[Epoch 5/6] [Batch 958/1166] [D loss: 0.034520, acc:  49%] [G loss: 11.040609] time: 4:53:53.902436
[Epoch 5/6] [Batch 959/1166] [D loss: 0.014266, acc:  49%] [G loss: 11.248816] time: 4:53:56.500467
[Epoch 5/6] [Batch 960/1166] [D loss: 0.014944, acc:  49%] [G loss: 11.121152] time: 4:53:59.098514
[Epoch 5/6] [Batch 961/1166] [D loss: 0.011008, acc:  49%] [G loss: 12.793405] time: 4:54:01.699559
[Epoch 5/6] [Batch 962/1166] [D loss: 0.076787, acc:  36%] [G loss: 9.605695] time: 4:54:04.297644
[Epoch 5/6] [Batch 963/1166] [D loss: 0.046939, acc:  46%] [G loss: 10.536999] time: 4:54:06.896663
[Epoch 5/6] [Batch 964/1166] [D loss: 0.033198, acc:  47%] [G loss: 11.880207] time: 4:54:09.487767
[Epoch 5/6] [Batch 965/1166] [D loss: 0.015755, acc:  49%] [G loss: 10.539473] time: 4:54:12.079804
[Epoch 5/6] [Batch 966/1166] [D loss: 0.011587, acc:  49%] [G loss: 11.029677] time: 4:54:14.674865
[Epoch 5/6] [Batch 967/1166] [D loss: 0.110078, acc:  30%] [G loss: 9.435592] time: 4:54:17.278903
[E

[Epoch 5/6] [Batch 1040/1166] [D loss: 0.038591, acc:  47%] [G loss: 9.979349] time: 4:57:26.777254
[Epoch 5/6] [Batch 1041/1166] [D loss: 0.085668, acc:  36%] [G loss: 9.031605] time: 4:57:29.369292
[Epoch 5/6] [Batch 1042/1166] [D loss: 0.014109, acc:  49%] [G loss: 11.054781] time: 4:57:31.968343
[Epoch 5/6] [Batch 1043/1166] [D loss: 0.039902, acc:  47%] [G loss: 9.790191] time: 4:57:34.564401
[Epoch 5/6] [Batch 1044/1166] [D loss: 0.078216, acc:  42%] [G loss: 9.793017] time: 4:57:37.157468
[Epoch 5/6] [Batch 1045/1166] [D loss: 0.011626, acc:  49%] [G loss: 9.645045] time: 4:57:39.758513
[Epoch 5/6] [Batch 1046/1166] [D loss: 0.009162, acc:  49%] [G loss: 9.615549] time: 4:57:42.348589
[Epoch 5/6] [Batch 1047/1166] [D loss: 0.009818, acc:  49%] [G loss: 11.029563] time: 4:57:44.941655
[Epoch 5/6] [Batch 1048/1166] [D loss: 0.010392, acc:  49%] [G loss: 10.077059] time: 4:57:47.539740
[Epoch 5/6] [Batch 1049/1166] [D loss: 0.034496, acc:  48%] [G loss: 10.935327] time: 4:57:50.172

[Epoch 5/6] [Batch 1122/1166] [D loss: 0.031114, acc:  48%] [G loss: 10.959834] time: 5:00:59.602172
[Epoch 5/6] [Batch 1123/1166] [D loss: 0.008870, acc:  49%] [G loss: 11.410154] time: 5:01:02.194273
[Epoch 5/6] [Batch 1124/1166] [D loss: 0.023008, acc:  48%] [G loss: 9.849606] time: 5:01:04.790300
[Epoch 5/6] [Batch 1125/1166] [D loss: 0.024097, acc:  49%] [G loss: 8.782451] time: 5:01:07.386359
[Epoch 5/6] [Batch 1126/1166] [D loss: 0.009802, acc:  49%] [G loss: 10.493724] time: 5:01:09.979426
[Epoch 5/6] [Batch 1127/1166] [D loss: 0.008876, acc:  49%] [G loss: 13.829147] time: 5:01:12.572492
[Epoch 5/6] [Batch 1128/1166] [D loss: 0.124776, acc:  25%] [G loss: 9.297933] time: 5:01:15.168583
[Epoch 5/6] [Batch 1129/1166] [D loss: 0.047901, acc:  45%] [G loss: 9.260472] time: 5:01:17.761618
[Epoch 5/6] [Batch 1130/1166] [D loss: 0.020411, acc:  48%] [G loss: 11.388842] time: 5:01:20.362695
[Epoch 5/6] [Batch 1131/1166] [D loss: 0.046111, acc:  46%] [G loss: 7.860362] time: 5:01:22.95

In [5]:
from scipy.misc import imsave
data = DataLoader(dataset_name="test40",
                                      img_res=(256, 256))
imgs_A, imgs_B = data.load_data(batch_size=40, is_testing=True)

fake_A = gan.generator.predict(imgs_B)

fake_A = 0.5 * fake_A + 0.5


os.makedirs('images/TUnetFullset', exist_ok=True)
f = 0
for img in fake_A:
    imsave('images/TUnetFullset/gen%d.png' % (f), img)
    f = f+1
    
    
    

C:\Users\miche\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [ ]:
from scipy.misc import imsave
data = DataLoader(dataset_name="val_256",
                                      img_res=(256, 256))
imgs_A, imgs_B = data.load_data(batch_size=60, is_testing=True)

r, c = 3, 3

fake_A = gan.generator.predict(imgs_B)

gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

# Rescale images 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5
fake_A = 0.5 * fake_A + 0.5
imgs_B = 0.5 * imgs_B + 0.5
imgs_A = 0.5 * imgs_A + 0.5
titles = ['Originale', 'B&W', 'Generata']


ic = 0
f=0
cnt = 0
for img in fake_A:
    f += 1
    imsave('images/test9000/testOutput%d.png' % (f), img)
    
    if ic==2:
        fig, axs = plt.subplots(r, c)
        axs[0, 0].set_title(titles[0])
        axs[0, 1].set_title(titles[1])
        axs[0, 2].set_title(titles[2])
        for i in range(r):
            axs[i,0].imshow(imgs_A[cnt])
            axs[i,1].imshow(imgs_B[cnt])
            axs[i,2].imshow(fake_A[cnt])
            cnt += 1
        
        fig.savefig("images/test9000/%d.png" % (cnt))
        plt.close()
    if ic == 2:
        ic=0
    else:
        ic += 1
    